In [62]:
import requests
import pandas as pd
from global_variables import *
import gzip

In [55]:
acs_variables = {
    'B01001_001E': 'total_population',
    'B01002_001E': 'median_age',
    'B11001_002E': 'no_of_households',
    'B25001_001E': 'housing_units',
    'B25003_001E': 'housing_units_occupancy_base',
    'B25003_002E': 'housing_units_occupancy_own',
    'B25003_003E': 'housing_units_occupancy_rent',
    'B25087_001E': 'housing_units_mortgage_base',
    'B25087_002E': 'housing_units_with_a_mortgage',
    'B19127_001E': 'aggregate_family_income',
    'B02001_001E': 'population_race_base',
    'B02001_002E': 'population_race_white',
    'B08303_001E': 'travel_time_to_work',
    'C16002_001E': 'households_language_base',
    'C16002_002E': 'households_language_english',
    'C16002_003E': 'households_language_spanish',
    'B25065_001E': 'aggregate_gross_rent',
    'B28011_001E': 'households_internet_base',
    'B28011_008E': 'households_internet_no',
    'B28011_002E': 'households_internet_yes',
    'B28003_001E': 'households_computer_base',
    'B28003_002E': 'households_computer_yes',
    'B28003_004E': 'households_computer_yes_bb',
    'B27010_001E': 'population_by_age_base',
    'B27010_002E': 'population_by_age_base_under_19',
    'B27010_051E': 'population_by_age_base_65_older',
    'B17010_001E': 'poverty_status_base',
    'B17010_002E': 'poverty_status_below',
    'B22010_001E': 'received_food_stamps_snap_base',
    'B22010_002E': 'received_food_stamps_snap_yes',
    'B19113_001E': 'median_family_income', #many missing values
    'B19001_014E': 'household_income_100_125k',
    'B19001_015E': 'household_income_125_150k',
    'B19001_016E': 'household_income_150_200k',
    'B19001_017E': 'household_income_200k_more',
    'B19001_002E': 'household_income_10k_15k',
    'B19001_003E': 'household_income_15k_19k',
    'B19001_004E': 'household_income_19k_25k',
    'B19001_005E': 'household_income_25k_30k',
    
}

get_vars = list(acs_variables.keys())

us_state_fips = [
    "01", "02", "04", "05", "06", "08", "09", "10", "12", "13",
    "15", "16", "17", "18", "19", "20", "21", "22", "23", "24",
    "25", "26", "27", "28", "29", "30", "31", "32", "33", "34",
    "35", "36", "37", "38", "39", "40", "41", "42", "44", "45",
    "46", "47", "48", "49", "50", "51", "53", "54", "55", "56"
]

base_url = 'https://api.census.gov/data/2021/acs/acs5'

predicates = {} 
predicates['get']=','.join(get_vars)
predicates['for']= 'block group:*'

predicates['key']=CENSUS_API_KEY

In [54]:
state="22"
predicates['in']='state:'+state+' county:033'
r=requests.get(base_url,params=predicates)
print(r)
json_data = r.json()
df = pd.DataFrame(json_data[1:], columns=json_data[0])
df.rename(columns=acs_variables, inplace=True)
print([col for col in df.columns if df.loc[0, col] is  None])

<Response [200]>
[]


In [ ]:
master_df = pd.DataFrame()

for state in us_state_fips:
    
    print(state)
    
    predicates['in']='state:'+state+' county:*'
    r=requests.get(base_url,params=predicates)

    json_data = r.json()
    df = pd.DataFrame(json_data[1:], columns=json_data[0])
    df.rename(columns=acs_variables, inplace=True)

    master_df = pd.concat([master_df, df], ignore_index=True)

In [63]:
output_file_path = "C:/Users/dratnadiwakara2/Documents/OneDrive - Louisiana State University/Raw Data/Dewey/acs_block_data.csv.gz" 
with gzip.open(output_file_path, 'wt', encoding='utf-8') as gzipped_file:
    master_df.to_csv(gzipped_file, index=False)